<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M0.532 · Pattern Recognition</p>
<p style="margin: 0; text-align:right;">Computational Engineering and Mathematics Master</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Computers, Multimedia and Telecommunications Department</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Feature Matching

For this notebook we need to do the install of the contrib modules from OpenCV. Contrib contains patended algorithms and others under development. SIFT and ORB algorithms are in contrib module

In [ ]:
!pip install opencv-contrib-python==4.4.0.44

In [ ]:
# import OpenCV library
import cv2

# we will use the following import to display images in colab:
from google.colab.patches import cv2_imshow

import numpy as np


In [ ]:
!wget https://github.com/opencv/opencv/blob/master/samples/data/leuvenA.jpg?raw=true -O leuvenA.jpg
!wget https://github.com/opencv/opencv/blob/master/samples/data/leuvenB.jpg?raw=true -O leuvenB.jpg

# read image
img1 = cv2.imread('leuvenA.jpg', cv2.IMREAD_COLOR)
img2 = cv2.imread('leuvenB.jpg', cv2.IMREAD_COLOR)

# convert image to grayscale
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)


In [ ]:
cv2_imshow(cv2.hconcat([img1, img2]))

Lets initiate SIFT Detector and use it with both images:

In [ ]:
# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with ORB
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

And plot the keypoints (same than feature detection):

In [ ]:
img1_sift = np.empty((img1.shape[0], img1.shape[1], 3), dtype=np.uint8)
img1_sift = cv2.drawKeypoints(img1, kp1, img1_sift)

img2_sift = np.empty((img2.shape[0], img2.shape[1], 3), dtype=np.uint8)
img2_sift = cv2.drawKeypoints(img2, kp1, img2_sift)

In [ ]:
cv2_imshow(cv2.hconcat([img1_sift, img2_sift]))

Once we have the features for both image we can start the feature matching

## Brute force

Lets do the matching with [BFMatcher](https://docs.opencv.org/4.5.4/d3/da1/classcv_1_1BFMatcher.html).

With BFMatcher.knnMatch() we get the k best matches. The algorithm takes the descriptor of one feature in the first image and compares it with all the features of the second image. The closest value (we have to define a distance measure also) is returned.


In [ ]:
# BFMatcher with default params
bf = cv2.BFMatcher()
matches = bf.match(des1,des2)

[Draw](https://docs.opencv.org/4.x/d4/d5d/group__features2d__draw.html) all matches between images:

In [ ]:
img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

In [ ]:
cv2_imshow(img3)

There are too many matches to see if the matching algorithm works well, lets try to keep only some matches (keep the best ones):

matches returned from the algorithm are a [DMatch Object](https://docs.opencv.org/3.4/d4/de0/classcv_1_1DMatch.html)

In [ ]:
# Sort the matches with the distance parameter of matches
matches = sorted(matches, key = lambda x:x.distance)

# Draw first 50 matches.
img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches[:50],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

In [ ]:
cv2_imshow(img3)

Best matches found by the algorithm looks mostly good. Not all the correspondences obtained are perfect but errors are in the neighborhod of the correct point

## Efficient matching

In the previous example we used a brute force implementation. Brute force is not feasible when working with large datasets. 

An algorithm for efficient matchin is FLANN (Fast Library for Approximate Nearest Neighbors). With FLANN we can compute the best matches in an optimized way, resulting in significant reductions on the computing time compared to BFMatcher

[FLANN](https://docs.opencv.org/4.x/dc/de2/classcv_1_1FlannBasedMatcher.html) needs as input a dictionary with the rellevant parameters for the matching search. We will use the same configuration than in the documentation [example](https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html): 

In [ ]:
# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params,search_params)

Lets compute the matches:

In [ ]:
matches = flann.match(des1,des2)


Lets keep only the best matches:

In [ ]:
# Sort the matches with the distance parameter of matches
matches = sorted(matches, key = lambda x:x.distance)

# Draw first 50 matches.
img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches[:50],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

In [ ]:
cv2_imshow(img3)

Results is similar than the one obtained with BFMatcher. Lets see if this algorithm is more efficient. We can install autotime that will tell the execution time of each cell:

In [ ]:
!pip install ipython-autotime

%load_ext autotime


In [ ]:
matches = bf.match(des1,des2)

In [ ]:
matches = flann.match(des1,des2)


The computation time of flann is 40% lower than the bfmatcher!